In [93]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
from sklearn.preprocessing import normalize
import pickle
from numpy import array
from sklearn.preprocessing import StandardScaler
import sys


%matplotlib inline


In [94]:

nome_arquivo = "Imagens/Validacao/IMG (2).bmp"

imagem = cv2.imread(nome_arquivo,0)
#plt.imshow(imagem)


In [95]:
#Colocando bordas na imagem
imagem = cv2.copyMakeBorder(imagem, 50, 50, 50, 50, cv2.BORDER_REPLICATE)
imagem_original = imagem

#Borra bastante
imagem = cv2.GaussianBlur(imagem,(75,75),0)
#plt.imshow(imagem, cmap='gray')

Limiar binary invertido + limiar de OTSU
o método de Otsu determina um valor de limite global ideal a partir do histograma da imagem

In [96]:
limiar, imagem_limiar = cv2.threshold(imagem,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU )

imagem_limiar_255 = imagem_limiar*255
imagem_limiar_255 = imagem_limiar_255.astype(np.uint8)
#plt.imshow(imagem_limiar_255, cmap='gray')

In [97]:

# Encontrando os contornos na imagem
contornos, void = cv2.findContours(imagem_limiar, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


Aqui temos o numero de objetos na imagem, que vai ser igual ao numero de bordas encontradas

In [98]:
size = len(contornos)
#size

In [99]:
aux = 1
df = pd.DataFrame()

for cnt in contornos:
    #area
    area = cv2.contourArea(cnt)
        
    #perimetro
    perimetro = cv2.arcLength(cnt,True)

    #compacidade
    compacidade = (perimetro**2)/area
    #print("Compacidade: ", compacidade)

    #circulatidade
    circularidade = (4*np.pi*area)/perimetro**2
    #print("circulatidade: ", circulatidade)


    #Cria eclipse envolvente 
    (xc,yc), (d1,d2), angle  = cv2.fitEllipse(cnt)

    #Pegas os pontos do eixo maior
    rmajor = max(d1,d2)/2
    if angle > 90:
        angle = angle - 90
    else:
        angle = angle + 90

    x1 = xc + math.cos(math.radians(angle))*rmajor
    y1 = yc + math.sin(math.radians(angle))*rmajor
    x2 = xc + math.cos(math.radians(angle+180))*rmajor
    y2 = yc + math.sin(math.radians(angle+180))*rmajor
    #calcula distancia
    eixo_maior = np.sqrt((x2-x1)**2 + (y2-y1)**2)

    #Pegas os pontos do eixo menor
    rminor = min(d1,d2)/2
    if angle > 90:
        angle = angle - 90
    else:
        angle = angle + 90

    x1 = xc + math.cos(math.radians(angle))*rminor
    y1 = yc + math.sin(math.radians(angle))*rminor
    x2 = xc + math.cos(math.radians(angle+180))*rminor
    y2 = yc + math.sin(math.radians(angle+180))*rminor
    #calcula distancia
    eixo_menor = np.sqrt((x2-x1)**2 + (y2-y1)**2)

    #Razão do eixo menor pelo eixo maior
    razao_eixos = eixo_menor/eixo_maior
    #print("Razão Eixos: ", razao_eixos)

    
    mask = np.zeros(imagem_original.shape,np.uint8)
    mask = cv2.drawContours(mask,[cnt],0,255,-1)
    #pixelpoints = np.transpose(np.nonzero(mask))
    
    mean_val = cv2.mean(imagem_original,mask = mask)
    auxiliar = mean_val[0]/255

    df = df.append({'Compacidade': compacidade, 'Circularidade': circularidade, 'Razão Eixos': razao_eixos,  'Intencidade Média': auxiliar}, ignore_index=True)

    imagem_contornos = cv2.drawContours(cv2.cvtColor(imagem_original, cv2.COLOR_BAYER_BG2BGRA), [cnt], 0, (222,0,0), 3)

    cv2.imwrite( str(aux) + ".bmp", imagem_contornos)
    #print("Intencidade media : ", razao_eixos)
    #print("")

    aux += 1

df.to_csv('teste.csv', index=False)

# Carregar modelo
with open('MLP/ClassificadorFolhas.pkl', 'rb') as f:
    model = pickle.load(f)

data = pd.read_csv('teste.csv')

X_normalized = normalize(data, axis=0)

sc=StandardScaler()
scaler = sc.fit(data)

testX_scaled = scaler.transform(data)

y_pred = model.predict(testX_scaled)
#print(y_pred)

C:\Users\bianc\AppData\Local\Temp\ipykernel_20356\3098652756.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Compacidade': compacidade, 'Circularidade': circularidade, 'Razão Eixos': razao_eixos,  'Intencidade Média': auxiliar}, ignore_index=True)
C:\Users\bianc\AppData\Local\Temp\ipykernel_20356\3098652756.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Compacidade': compacidade, 'Circularidade': circularidade, 'Razão Eixos': razao_eixos,  'Intencidade Média': auxiliar}, ignore_index=True)
C:\Users\bianc\AppData\Local\Temp\ipykernel_20356\3098652756.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Compacidade': compacidade, 'Circularidade': circularidade, 'Ra

In [100]:
F = 0
A = 0
Q = 0
P = 0
C = 0
U = 0

for x in y_pred:
    if x == 0:
        F += 1
    if x == 1:
        A += 1
    if x == 2:
        Q += 1
    if x == 3:
        P += 1
    if x == 4:
        C += 1
    if x == 5:
        U += 1

print("Arquivo: ", nome_arquivo)
print("Numero de folhas: ", size)
if F > 0:
    print("Folhado (", F,")")
if A > 0:
    print("Araçá (", A,")")
if Q > 0:
    print("Quaresmeira (", Q,")")
if P > 0:
    print("Pessegueiro (", P,")")
if C > 0:
    print("Coleus (", C,")")
if U > 0:
    print("Uva do mato (", U,")")

Arquivo:  Imagens/Validacao/IMG (2).bmp
Numero de folhas:  8
Folhado ( 2 )
Araçá ( 2 )
Pessegueiro ( 1 )
Coleus ( 1 )
Uva do mato ( 2 )
